# Web scraper to extract intial specifications and product links

## DRAFT - currently under development

### Important Details:
Website: https://www.appliancesonline.com.au/

In [ ]:
# Import modules
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

from time import sleep
import pandas as pd
import pickle

In [ ]:
# find element : Helper function
def find_element(element, element_tag, element_type):
    result = None
    try:
        if(element_type == 'css_selector'):
            result = element.find_element_by_css_selector(element_tag).text
            
        elif (element_type == 'href'):
            result = element.find_element_by_class_name(element_tag).get_attribute('href')
            
    except NoSuchElementException:
        result = None
    
    return(result)   

In [ ]:
# get_product_data : Helper function
def get_product_data(driver, data_dict):
    Cells = driver.find_elements_by_css_selector('aol-product[trackinglistname="grid"] div.inner')
    offset = len(data_dict)
    for i, cell in enumerate(Cells):
            
        # Convert data to a dictionary
        data_dict[i+offset] = {"ID": str(i+offset).zfill(5),
                               "Product_Name": find_element(cell, 'div.product-name', 'css_selector'),                                                     
                               "Product_Link": find_element(cell, 'body-link', 'href'),
                               "Price": find_element(cell, 'div.price', 'css_selector'),
                               "Review_Count": find_element(cell, 'span.label', 'css_selector'),
                               "Review_Score": find_element(cell, 'span.avg-rating', 'css_selector'),
                               "Original_Price": find_element(cell, 'div.amount', 'css_selector')}

    return(data_dict)

In [ ]:
# openPage : Helper function
def openPage(url_prefix, url_suffix, driver):
    product_data= {}
    
    # Page to load
    page_to_load = url_prefix + '1' + url_suffix
    
    # Load the web page
    driver.get(page_to_load)
    sleep(2)
    
    # Identify the number of products
    product_count = int(driver.find_element_by_css_selector('div.products-bar div.products-count').text.split(" ")[0])
    print(f'There were {product_count} products detected')
    
    #Get data from the first page 
    product_data = get_product_data(driver, data_dict = product_data)
    
    # Get data from the rest of the pages
    for i in range(2,int((product_count/24)+2)):
        sleep(2)
        page_to_load = url_prefix + str(i) + url_suffix
        driver.get(page_to_load)
        sleep(2)
        product_data = get_product_data(driver, data_dict = product_data)            
    return(product_data)


In [ ]:
# save_DataFrame : helper function
def save_DataFrame(df, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# load_DataFrame : helper function     
def load_DataFrame(filename):
    with open(filename, 'rb') as handle:
        df = pickle.load(handle)
        
    return(df)

In [ ]:
# Main program

# Product page that lists the products
page_prefix = 'https://www.appliancesonline.com.au/filter/consumer-electronics/tvs/?currentpage='
page_suffix = '&sortkey=highestrated'  

# incognito mode
chrome_options = Options()
chrome_options.add_argument('--incognito')
driverPATH = 'C:\\University\\Master Class\\chromedriver_win32\\chromedriver.exe'
chrome_Browser = webdriver.Chrome(driverPATH, options=chrome_options)

# Get the product names and links and convert to a dataframe
TV_dict = openPage(page_prefix, page_suffix, chrome_Browser)
TV_df = pd.DataFrame(TV_dict).T

#Close the browser
chrome_Browser.close()

# Save the TV_df to disk
save_DataFrame(df = TV_df, filename='TV_names_and_links_df.pickle')

# Display the TV_df
TV_df.head(3)